In [8]:
from bs4 import BeautifulSoup as bs
import requests

r = requests.get("https://en.wikipedia.org/wiki/Toy_Story_3")

#Convert to a beautiful soup object
soup = bs(r.content)

#Print out the HTML

contents = soup.prettify()
#print(contents)

#this is how we would capture info from the website using requests and beautiful soup and then further focus on the table rows (tr)

info_box = soup.find(class_="infobox vevent")
info_rows = info_box.find_all("tr")
for row in info_rows:
    print (row.prettify())


#creating a function get_content_value that handles list and regular texts from the table data (td)
#running a for loop through the table rows (stored inside info_rows defined above) and:
#title (index 0) is received by defining and then picture is skipped at index 1 and the rest is automated 
#string replacement is used to repalce weird haracters with space
#get_text method's properties were used to cleaup spacing issues (refer to documentation)

def get_content_value(row_data):
    if row_data.find("li"):
        return [li.get_text(" ", strip=True).replace("\xa0"," ") for li in row_data.find_all("li")]
    else:
        return row_data.get_text(" ", strip=True).replace("\xa0"," ")
    
movie_info = {}

for index, row in enumerate(info_rows):
    if index == 0:
        movie_info['title'] = row.find("th").get_text(" ", strip=True)
    elif index == 1:
        continue
    else:
        content_key = row.find("th").get_text(" ", strip=True)
        content_value = get_content_value(row.find("td"))
        movie_info[content_key] = content_value

movie_info

### Lets begin by scraping the list of Disney movies

In [9]:
rr = requests.get("https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films")

# Convert to a beautiful soup object
ssoup = bs(r.content)

# Print out the HTML

ccontents = ssoup.prettify()

NameError: name 'r' is not defined

In [33]:
#this time we are going to use a different method called select instead of find_all to work with CSS selectors

mmovies = ssoup.select(".wikitable.sortable i" )# i represents everything written in etalics within the table

mmovies[0:10]

[<i><a href="/wiki/Academy_Award_Review_of_Walt_Disney_Cartoons" title="Academy Award Review of Walt Disney Cartoons">Academy Award Review of Walt Disney Cartoons</a></i>,
 <i><a href="/wiki/Snow_White_and_the_Seven_Dwarfs_(1937_film)" title="Snow White and the Seven Dwarfs (1937 film)">Snow White and the Seven Dwarfs</a></i>,
 <i><a href="/wiki/Pinocchio_(1940_film)" title="Pinocchio (1940 film)">Pinocchio</a></i>,
 <i><a href="/wiki/Fantasia_(1940_film)" title="Fantasia (1940 film)">Fantasia</a></i>,
 <i><a href="/wiki/The_Reluctant_Dragon_(1941_film)" title="The Reluctant Dragon (1941 film)">The Reluctant Dragon</a></i>,
 <i><a href="/wiki/Dumbo" title="Dumbo">Dumbo</a></i>,
 <i><a href="/wiki/Bambi" title="Bambi">Bambi</a></i>,
 <i><a href="/wiki/Saludos_Amigos" title="Saludos Amigos">Saludos Amigos</a></i>,
 <i><a href="/wiki/Victory_Through_Air_Power_(film)" title="Victory Through Air Power (film)">Victory Through Air Power</a></i>,
 <i><a href="/wiki/The_Three_Caballeros" title=

In [34]:
#lets seperately catch the href and title from the items

mmovies[1].a["href"]

'/wiki/Snow_White_and_the_Seven_Dwarfs_(1937_film)'

In [35]:
mmovies[1].a["title"]

'Snow White and the Seven Dwarfs (1937 film)'

### Lets use what we figured in above markdown and code to define functions that are going to automate things for us

In [13]:
def get_content_value(row_data):
    if row_data.find("li"):
        return [li.get_text(" ", strip=True).replace("\xa0"," ") for li in row_data.find_all("li")]
    elif row_data.find("br"):
        return [text for text in row_data.stripped_strings]# this is to ensure anything that has br will be joined
    else:
        return row_data.get_text(" ", strip=True).replace("\xa0"," ")

def clean_tags(soup):
    for tag in soup.find_all("sup"):
        tag.decompose()
    
def get_info_box(url):

    r = requests.get(url)
    
    soup = bs(r.content)
    info_box = soup.find(class_="infobox vevent")
    info_rows = info_box.find_all("tr")
    
    clean_tags(soup)

    movie_info = {}
    for index, row in enumerate(info_rows):
        if index == 0:
            movie_info['title'] = row.find("th").get_text(" ", strip=True)
        else:
            header = row.find("th")
            if header:
                content_key = row.find("th").get_text(" ", strip=True)
                content_value = get_content_value(row.find("td"))
                movie_info[content_key] = content_value
            
    return movie_info

In [16]:
get_info_box("https://en.wikipedia.org/wiki/Howl%27s_Moving_Castle_(film)")

{'title': "Howl's Moving Castle",
 'Japanese': 'ハウルの動く城',
 'Hepburn': 'Hauru no Ugoku Shiro',
 'Directed by': 'Hayao Miyazaki',
 'Screenplay by': 'Hayao Miyazaki',
 'Based on': ["Howl's Moving Castle", 'by', 'Diana Wynne Jones'],
 'Produced by': 'Toshio Suzuki',
 'Starring': ['Chieko Baisho', 'Takuya Kimura', 'Akihiro Miwa'],
 'Cinematography': 'Atsushi Okui',
 'Edited by': 'Takeshi Seyama',
 'Music by': 'Joe Hisaishi',
 'Production company': 'Studio Ghibli',
 'Distributed by': 'Toho',
 'Release date': ['5 September 2004 ( 2004-09-05 ) ( Venice )',
  '20 November 2004 ( 2004-11-20 ) (Japan)'],
 'Running time': '119 minutes',
 'Country': 'Japan',
 'Language': 'Japanese',
 'Budget': ['¥', '2.4 billion', 'US$24 million'],
 'Box office': ['¥', '23.2 billion', 'US$236 million (worldwide)']}

In [17]:
r = requests.get("https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films")
soup = bs(r.content)
movies = soup.select(".wikitable.sortable i a")#grabbing the a item out of the italics to handle errors with no links

base_path = "https://en.wikipedia.org/"

movie_info_list = []
for index, movie in enumerate(movies):
    #if index == 10: # this is how you check for errors without running everthing
        #break    
    if index % 10 == 0: # just an indexed way to check for errors; pretty helpful
        print(index)
    try:
        relative_path = movie['href']
        full_path = base_path + relative_path
        title = movie['title']
        
        movie_info_list.append(get_info_box(full_path))
        
    except Exception as e:
        print(movie.get_text())
        print(e)

0
10
20
30
40
Zorro the Avenger
'NoneType' object has no attribute 'find'
The Sign of Zorro
'NoneType' object has no attribute 'find'
50
60
70
80
90
100
110
120
True-Life Adventures
'NoneType' object has no attribute 'find_all'
130
140
The London Connection
'NoneType' object has no attribute 'find'
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
Better Nate Than Never
'NoneType' object has no attribute 'find_all'
Sister Act 3
'NoneType' object has no attribute 'find'
460
Tron: Ares
'NoneType' object has no attribute 'find'
470


In [18]:
len(movie_info_list)

466

### Save/Reload Movie Data

In [191]:
import json

def save_data(title, data):
    with open(title, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

In [192]:
import json

def load_data(title):
    with open(title, encoding="utf-8") as f:
        return json.load(f)

In [21]:
save_data("disney_data_clean.json", movie_info_list)

### Lets clean the data

In [28]:
movie_info_list = load_data("disney_data_clean.json")

#### After going through the data, the following needs to be done
- Clean up the references (by defining a new fuction called clean_tags)
- Convert running time into an integer 
- Convert dates into datetime object
- Split up the long strings (by editing the get_content_value function to handle the br tags)
- Convert Budget & Box office to numbers
- Address NoneType errors from generating at the scrappign level (by editing the get_info_box function to only look at headers after index 0)

#### Lets fix the running time to only have the integer

In [23]:
print([movie.get('Running time', 'N/A') for movie in movie_info_list])

['41 minutes (74 minutes 1966 release)', '83 minutes', '88 minutes', '126 minutes', '74 minutes', '64 minutes', '70 minutes', '42 minutes', '70 min', '71 minutes', '75 minutes', '94 minutes', '73 minutes', '75 minutes', '82 minutes', '68 minutes', '74 minutes', '96 minutes', '75 minutes', '84 minutes', '77 minutes', '92 minutes', '69 minutes', '81 minutes', ['60 minutes (VHS version)', '71 minutes (original)'], '127 minutes', '92 minutes', '76 minutes', '75 minutes', '73 minutes', '85 minutes', '81 minutes', '70 minutes', '90 min.', '80 minutes', '75 minutes', '83 minutes', '83 minutes', '72 minutes', '97 minutes', '75 minutes', '104 minutes', '93 minutes', '105 minutes', '95 minutes', '97 minutes', '134 minutes', '69 minutes', '92 minutes', '131 minutes', '79 minutes', '97 minutes', '128 minutes', '74 minutes', '91 minutes', '105 minutes', '98 minutes', '130 minutes', '89 min.', '93 minutes', '67 minutes', '98 minutes', '100 minutes', '118 minutes', '103 Minutes', '110 minutes', '80 m

In [26]:
def minutes_to_integer(running_time):
    if running_time == "N/A": # this is simply handlign the N/As
        return None
    
    if isinstance(running_time, list): # this one checks if its a list (isinstance seems quite useful)
        return int(running_time[0].split(" ")[0])
    else: # is a string then spliting by whitespace and grabbign the first index
        return int(running_time.split(" ")[0])

In [32]:
minutes_to_integer(["67 minutes", "Hola"])

67

In [29]:
# now to ad a new key value to the json

for movie in movie_info_list:
    movie['Running time (int)'] = minutes_to_integer(movie.get('Running time', "N/A"))

In [33]:
movie_info_list[-10]

{'title': 'The Rocketeer',
 'Directed by': 'Joe Johnston',
 'Screenplay by': ['Danny Bilson', 'Paul De Meo'],
 'Story by': ['Danny Bilson', 'Paul De Meo', 'William Dear'],
 'Based on': ['The Rocketeer', 'by', 'Dave Stevens'],
 'Produced by': ['Charles Gordon', 'Lawrence Gordon', 'Lloyd Levin'],
 'Starring': ['Bill Campbell',
  'Alan Arkin',
  'Jennifer Connelly',
  'Paul Sorvino',
  'Timothy Dalton'],
 'Cinematography': 'Hiro Narita',
 'Edited by': ['Michael A. Stevenson', 'Arthur Schmidt'],
 'Music by': 'James Horner',
 'Production companies': ['Walt Disney Pictures',
  'Touchstone Pictures',
  'Silver Screen Partners IV',
  'Gordon Company'],
 'Distributed by': 'Buena Vista Pictures Distribution',
 'Release date': ['June 21, 1991 ( 1991-06-21 )'],
 'Running time': '108 minutes',
 'Country': 'United States',
 'Language': 'English',
 'Budget': '$35-40 million',
 'Box office': '$46.7 million (USA)',
 'Running time (int)': 108}

#### Let's fix the Budget Value now

In [34]:
print([movie.get('Budget', 'N/A') for movie in movie_info_list])

['N/A', '$1.49 million', '$2.6 million', '$2.28 million', '$600,000', '$950,000', '$858,000', 'N/A', '$788,000', 'N/A', '$1.35 million', '$2.125 million', 'N/A', '$1.5 million', '$1.5 million', 'N/A', '$2.2 million', '$1,800,000', '$3 million', 'N/A', '$4 million', '$2 million', '$300,000', '$1.8 million', 'N/A', '$5 million', 'N/A', '$4 million', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', '$700,000', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', '$6 million', 'under $1 million or $1,250,000', 'N/A', '$2 million', 'N/A', 'N/A', '$2.5 million', 'N/A', 'N/A', '$4 million', '$3.6 million', 'N/A', 'N/A', 'N/A', 'N/A', '$3 million', 'N/A', '$3 million', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', '$3 million', 'N/A', 'N/A', 'N/A', 'N/A', '$4.4–6 million', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', '$4 million', 'N/A', '$5 million', 'N/A', 'N/A', 'N/A', 'N/A', '$5 million', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', '$4 million', 'N/A', 'N/A', 'N/A', '

In [160]:
import re

amounts = r"thousand|million|billion"
number = r"\d+(,\d{3})*\.*\d*"

word_re = rf"\${number}(-|\sto\s|–)?({number})?\s({amounts})"
value_re = rf"\${number}"

def word_to_value(word):
    value_dict = {"thousand": 1000, "million": 1000000, "billion": 1000000000}
    return value_dict[word]

def parse_word_syntax(string):
    value_string = re.search(number, string).group()
    value = float(value_string.replace(",",""))
    word = re.search(amounts, string, flags=re.I).group().lower()
    word_value = word_to_value(word)
    return value*word_value

def parse_value_syntax(string):
    value_string = re.search(number, string).group()
    value = float(value_string.replace(",",""))
    return value

def money_conversion(money):

    if money == "N/A":
        return None
    
    if isinstance(money, list):
        money = money[0]
    
    word_syntax = re.search(word_re, money, flags=re.I)
    value_syntax = re.search(value_re, money)

    if word_syntax:
        return parse_word_syntax(word_syntax.group())

    elif value_syntax:
        return parse_value_syntax(value_syntax.group()) 
    
    else:
        return None

In [161]:
# now to ad a new key value to the json

for movie in movie_info_list:
    movie['Budget (float)'] = money_conversion(movie.get('Budget', "N/A"))
    movie['Box office (float)'] = money_conversion(movie.get('Box office', "N/A"))

In [61]:
movie_info_list.index('title': 'Beauty and the Beast')

SyntaxError: invalid syntax (<ipython-input-61-8738ac90232d>, line 1)

In [62]:
print([movie.get('title', 'Beauty and the Beast') for movie in movie_info_list])

['Academy Award Review of Walt Disney Cartoons', 'Snow White and the Seven Dwarfs', 'Pinocchio', 'Fantasia', 'The Reluctant Dragon', 'Dumbo', 'Bambi', 'Saludos Amigos', 'Victory Through Air Power', 'The Three Caballeros', 'Make Mine Music', 'Song of the South', 'Fun and Fancy Free', 'Melody Time', 'So Dear to My Heart', 'The Adventures of Ichabod and Mr. Toad', 'Cinderella', 'Treasure Island', 'Alice in Wonderland', 'The Story of Robin Hood', 'Peter Pan', 'The Sword and the Rose', 'The Living Desert', 'Rob Roy: The Highland Rogue', 'The Vanishing Prairie', '20,000 Leagues Under the Sea', 'Davy Crockett: King of the Wild Frontier', 'Lady and the Tramp', 'The African Lion', 'The Littlest Outlaw', 'The Great Locomotive Chase', 'Davy Crockett and the River Pirates', 'Secrets of Life', 'Westward Ho the Wagons!', 'Johnny Tremain', 'Perri', 'Old Yeller', 'The Light in the Forest', 'White Wilderness', 'Tonka', 'Sleeping Beauty', 'The Shaggy Dog', "Darby O'Gill and the Little People", 'Third Ma

In [75]:
movie_info_list[-68]

{'title': 'Beauty and the Beast',
 'Directed by': 'Bill Condon',
 'Screenplay by': ['Stephen Chbosky', 'Evan Spiliotopoulos'],
 'Based on': ["Disney 's Beauty and the Beast by Linda Woolverton",
  'Beauty and the Beast by Jeanne-Marie Leprince de Beaumont'],
 'Produced by': ['David Hoberman', 'Todd Lieberman'],
 'Starring': ['Emma Watson',
  'Dan Stevens',
  'Luke Evans',
  'Kevin Kline',
  'Josh Gad',
  'Ewan McGregor',
  'Stanley Tucci',
  'Audra McDonald',
  'Gugu Mbatha-Raw',
  'Ian McKellen',
  'Emma Thompson'],
 'Cinematography': 'Tobias A. Schliessler',
 'Edited by': 'Virginia Katz',
 'Music by': 'Alan Menken',
 'Production companies': ['Walt Disney Pictures', 'Mandeville Films'],
 'Distributed by': ['Walt Disney Studios', 'Motion Pictures'],
 'Release date': ['February 23, 2017 ( 2017-02-23 ) ( Spencer House )',
  'March 17, 2017 ( 2017-03-17 ) (United States)'],
 'Running time': '129 minutes',
 'Country': 'United States',
 'Language': 'English',
 'Budget': '$160–255 million',


In [159]:
money_conversion(movie_info_list[-68]['Budget'])

160000000.0

#### Convert dates to datetimes

In [163]:
from datetime import datetime
    
dates = [movie.get('Release date', 'N/A') for movie in movie_info_list]

def clean_date(date):
    return date.split("(")[0].strip()

def date_conversion(date):
    if isinstance(date, list):
        date = date[0]
        
    if date == "N/A":
        return None
    
    date_str = clean_date(date)
    fmts = ["%B %d, %Y", "%d %B %Y"]
    for fmt in fmts:
        try:
            return datetime.strptime(date_str, fmt)
        except:
            pass
    return None
    
for date in dates:
    print(date_conversion(date))
    print()

1937-05-19 00:00:00

1937-12-21 00:00:00

1940-02-07 00:00:00

1940-11-13 00:00:00

1941-06-27 00:00:00

1941-10-23 00:00:00

1942-08-09 00:00:00

1942-08-24 00:00:00

1943-07-17 00:00:00

1944-12-21 00:00:00

1946-04-20 00:00:00

1946-11-12 00:00:00

1947-09-27 00:00:00

1948-05-27 00:00:00

1948-11-29 00:00:00

1949-10-05 00:00:00

1950-02-15 00:00:00

1950-06-22 00:00:00

1951-07-26 00:00:00

1952-03-13 00:00:00

1953-02-05 00:00:00

1953-07-23 00:00:00

1953-11-10 00:00:00

1953-10-26 00:00:00

1954-08-17 00:00:00

1954-12-23 00:00:00

1955-05-25 00:00:00

1955-06-22 00:00:00

1955-09-14 00:00:00

1955-12-22 00:00:00

1956-06-08 00:00:00

1956-07-18 00:00:00

1956-09-04 00:00:00

1956-12-20 00:00:00

1957-06-19 00:00:00

1957-08-28 00:00:00

1957-12-25 00:00:00

1958-07-08 00:00:00

1958-08-12 00:00:00

1958-12-25 00:00:00

1959-01-29 00:00:00

1959-03-19 00:00:00

1959-06-24 00:00:00

1959-11-10 00:00:00

1960-01-21 00:00:00

1960-02-24 00:00:00

1960-05-19 00:00:00

None

1960-11

In [166]:
for movie in movie_info_list:
    movie['Release date (datetime)'] = date_conversion(movie.get('Release date', 'N/A'))

In [93]:
movie_info_list[50]

{'title': 'One Hundred and One Dalmatians',
 'Directed by': ['Clyde Geronimi', 'Hamilton Luske', 'Wolfgang Reitherman'],
 'Story by': 'Bill Peet',
 'Based on': ['The Hundred and One Dalmatians', 'by', 'Dodie Smith'],
 'Produced by': 'Walt Disney',
 'Starring': ['Rod Taylor',
  'Cate Bauer',
  'Betty Lou Gerson',
  'Ben Wright',
  'Bill Lee (singing voice)',
  'Lisa Davis',
  'Martha Wentworth'],
 'Edited by': ['Roy M. Brewer, Jr.', 'Donald Halliday'],
 'Music by': 'George Bruns',
 'Production company': 'Walt Disney Productions',
 'Distributed by': 'Buena Vista Distribution',
 'Release date': ['January 25, 1961 ( 1961-01-25 )'],
 'Running time': '79 minutes',
 'Country': 'United States',
 'Language': 'English',
 'Budget': '$3.6 million',
 'Box office': '$303 million',
 'Running time (int)': 79,
 'Budget (float)': 3600000.0,
 'Box office (float)': 303000000.0,
 'Release date (datetime)': datetime.datetime(1961, 1, 25, 0, 0)}

#### Since datetime object is not recognizable in json, let's use pickle to save it

In [168]:
import pickle

def save_data_pickle(name, data):
    with open(name, 'wb') as f:
        pickle.dump(data, f)

In [169]:
import pickle

def load_data_pickle(name):
    with open(name, 'rb') as f:
        return pickle.load(f)

In [170]:
save_data_pickle("disney_movie_data_more_cleaned.pickle", movie_info_list)

In [171]:
a = load_data_pickle("disney_movie_data_more_cleaned.pickle")

In [148]:
a[5]

{'title': 'Dumbo',
 'Directed by': ['Ben Sharpsteen',
  'Norman Ferguson',
  'Wilfred Jackson',
  'Bill Roberts',
  'Jack Kinney',
  'Samuel Armstrong'],
 'Story by': ['Otto Englander', 'Joe Grant', 'Dick Huemer'],
 'Based on': ['Dumbo, the Flying Elephant',
  'by',
  'Helen Aberson',
  'Harold Pearl'],
 'Produced by': 'Walt Disney',
 'Starring': ['Edward Brophy',
  'Verna Felton',
  'Cliff Edwards',
  'Herman Bing',
  'Sterling Holloway',
  'Margaret Wright',
  'Hall Johnson Choir'],
 'Narrated by': 'John McLeish',
 'Music by': ['Frank Churchill', 'Oliver Wallace'],
 'Production company': 'Walt Disney Productions',
 'Distributed by': 'RKO Radio Pictures',
 'Release date': ['October 23, 1941 ( 1941-10-23 ) (New York City)',
  'October 31, 1941 ( 1941-10-31 ) (U.S.)'],
 'Running time': '64 minutes',
 'Country': 'United States',
 'Language': 'English',
 'Budget': '$950,000',
 'Box office': '>$1.3 million (est. United States/Canada rentals, 1941)',
 'Running time (int)': 64,
 'Budget (flo

In [172]:
a == movie_info_list

True

In [173]:
movie_info_list = load_data_pickle('disney_movie_data_more_cleaned.pickle')

### Atttach IMDB and Rotten Tomatoes/Metascore Score

In [151]:
# http://www.omdbapi.com/?apikey=[yourkey]&

In [175]:
import requests
import urllib
import os 

def get_omdb_info(title):
    base_url = "http://www.omdbapi.com/?" # a ? is added to indicate that we are starting parameters
    parameters = {"apikey": "your unique key would go here", 't': title} 
    params_encoded = urllib.parse.urlencode(parameters)
    full_url = base_url + params_encoded
    return requests.get(full_url).json()

def get_rotten_tomato_score(omdb_info):
    ratings = omdb_info.get('Ratings', []) # the comma denotes othersise which is an empty list, in this case
    for rating in ratings:
        # print(rating) - this would be to see every item under rating
        if rating['Source'] == 'Rotten Tomatoes':
            return rating ['Value']
    return None
    
info = get_omdb_info('into the woods')
get_rotten_tomato_score(info)
    

'71%'

In [176]:
lion_king = get_omdb_info('lion king')

In [177]:
for key in lion_king:
    print(key, ' : ', lion_king[key])

Title  :  The Lion King
Year  :  1994
Rated  :  G
Released  :  24 Jun 1994
Runtime  :  88 min
Genre  :  Animation, Adventure, Drama
Director  :  Roger Allers, Rob Minkoff
Writer  :  Irene Mecchi, Jonathan Roberts, Linda Woolverton
Actors  :  Matthew Broderick, Jeremy Irons, James Earl Jones
Plot  :  Lion prince Simba and his father are targeted by his bitter uncle, who wants to ascend the throne himself.
Language  :  English, Swahili, Xhosa, Zulu
Country  :  United States
Awards  :  Won 2 Oscars. 39 wins & 35 nominations total
Poster  :  https://m.media-amazon.com/images/M/MV5BYTYxNGMyZTYtMjE3MS00MzNjLWFjNmYtMDk3N2FmM2JiM2M1XkEyXkFqcGdeQXVyNjY5NDU4NzI@._V1_SX300.jpg
Ratings  :  [{'Source': 'Internet Movie Database', 'Value': '8.5/10'}, {'Source': 'Rotten Tomatoes', 'Value': '93%'}, {'Source': 'Metacritic', 'Value': '88/100'}]
Metascore  :  88
imdbRating  :  8.5
imdbVotes  :  981,398
imdbID  :  tt0110357
Type  :  movie
DVD  :  04 Oct 2011
BoxOffice  :  $422,783,777
Production  :  Walt D

In [178]:
lion_king.get('Type')

'movie'

In [179]:
lion_king.get('Ratings')

[{'Source': 'Internet Movie Database', 'Value': '8.5/10'},
 {'Source': 'Rotten Tomatoes', 'Value': '93%'},
 {'Source': 'Metacritic', 'Value': '88/100'}]

In [180]:
for movie in movie_info_list:
    title = movie['title']
    omdb_info = get_omdb_info(title)
    movie['imdb'] = omdb_info.get('imdbRating', None) # find key, otherwise None so that the loop doesn't break
    movie['metascore'] = omdb_info.get('Metascore', None)
    movie['rotten_tomatoes'] = get_rotten_tomato_score(omdb_info)

In [181]:
movie_info_list[-100]

{'title': 'The Lone Ranger',
 'Directed by': 'Gore Verbinski',
 'Screenplay by': ['Justin Haythe', 'Ted Elliott', 'Terry Rossio'],
 'Story by': ['Ted Elliott', 'Terry Rossio', 'Justin Haythe'],
 'Based on': ['Fran Striker', 'George W. Trendle'],
 'Produced by': ['Jerry Bruckheimer', 'Gore Verbinski'],
 'Starring': ['Johnny Depp',
  'Armie Hammer',
  'Tom Wilkinson',
  'William Fichtner',
  'Barry Pepper',
  'James Badge Dale',
  'Helena Bonham Carter'],
 'Cinematography': 'Bojan Bazelli',
 'Edited by': ['James Haygood', 'Craig Wood'],
 'Music by': 'Hans Zimmer',
 'Production companies': ['Walt Disney Pictures',
  'Jerry Bruckheimer Films',
  'Infinitum Nihil',
  'Blind Wink Productions'],
 'Distributed by': ['Walt Disney Studios', 'Motion Pictures'],
 'Release date': ['June 22, 2013 ( 2013-06-22 ) ( Hyperion Theatre )',
  'July 3, 2013 ( 2013-07-03 ) (United States)'],
 'Running time': '149 minutes',
 'Country': 'United States',
 'Language': 'English',
 'Budget': '$225–250 million',
 '

In [182]:
save_data_pickle('disney_movie_data_final.pickle', movie_info_list)

### Save data as JSON and CSV

In [184]:
movie_info_list[50]

{'title': 'One Hundred and One Dalmatians',
 'Directed by': ['Clyde Geronimi', 'Hamilton Luske', 'Wolfgang Reitherman'],
 'Story by': 'Bill Peet',
 'Based on': ['The Hundred and One Dalmatians', 'by', 'Dodie Smith'],
 'Produced by': 'Walt Disney',
 'Starring': ['Rod Taylor',
  'Cate Bauer',
  'Betty Lou Gerson',
  'Ben Wright',
  'Bill Lee (singing voice)',
  'Lisa Davis',
  'Martha Wentworth'],
 'Edited by': ['Roy M. Brewer, Jr.', 'Donald Halliday'],
 'Music by': 'George Bruns',
 'Production company': 'Walt Disney Productions',
 'Distributed by': 'Buena Vista Distribution',
 'Release date': ['January 25, 1961 ( 1961-01-25 )'],
 'Running time': '79 minutes',
 'Country': 'United States',
 'Language': 'English',
 'Budget': '$3.6 million',
 'Box office': '$303 million',
 'Running time (int)': 79,
 'Budget (float)': 3600000.0,
 'Box office (float)': 303000000.0,
 'Release date (datetime)': datetime.datetime(1961, 1, 25, 0, 0),
 'imdb': '7.3',
 'metascore': '83',
 'rotten_tomatoes': '98%'}

In [187]:
movie_info_copy = [movie.copy() for movie in movie_info_list] # making a copy

In [188]:
movie_info_copy[50]

{'title': 'One Hundred and One Dalmatians',
 'Directed by': ['Clyde Geronimi', 'Hamilton Luske', 'Wolfgang Reitherman'],
 'Story by': 'Bill Peet',
 'Based on': ['The Hundred and One Dalmatians', 'by', 'Dodie Smith'],
 'Produced by': 'Walt Disney',
 'Starring': ['Rod Taylor',
  'Cate Bauer',
  'Betty Lou Gerson',
  'Ben Wright',
  'Bill Lee (singing voice)',
  'Lisa Davis',
  'Martha Wentworth'],
 'Edited by': ['Roy M. Brewer, Jr.', 'Donald Halliday'],
 'Music by': 'George Bruns',
 'Production company': 'Walt Disney Productions',
 'Distributed by': 'Buena Vista Distribution',
 'Release date': ['January 25, 1961 ( 1961-01-25 )'],
 'Running time': '79 minutes',
 'Country': 'United States',
 'Language': 'English',
 'Budget': '$3.6 million',
 'Box office': '$303 million',
 'Running time (int)': 79,
 'Budget (float)': 3600000.0,
 'Box office (float)': 303000000.0,
 'Release date (datetime)': datetime.datetime(1961, 1, 25, 0, 0),
 'imdb': '7.3',
 'metascore': '83',
 'rotten_tomatoes': '98%'}

#### Make the datetime object serialized so that it can be stored in a JSON

In [189]:
for movie in movie_info_copy:
    current_date = movie['Release date (datetime)']
    if current_date:
        movie['Release date (datetime)'] = current_date.strftime("%B %d, %Y")
    else:
        movie['Release date (datetime)'] = None

In [190]:
movie_info_copy[50]

{'title': 'One Hundred and One Dalmatians',
 'Directed by': ['Clyde Geronimi', 'Hamilton Luske', 'Wolfgang Reitherman'],
 'Story by': 'Bill Peet',
 'Based on': ['The Hundred and One Dalmatians', 'by', 'Dodie Smith'],
 'Produced by': 'Walt Disney',
 'Starring': ['Rod Taylor',
  'Cate Bauer',
  'Betty Lou Gerson',
  'Ben Wright',
  'Bill Lee (singing voice)',
  'Lisa Davis',
  'Martha Wentworth'],
 'Edited by': ['Roy M. Brewer, Jr.', 'Donald Halliday'],
 'Music by': 'George Bruns',
 'Production company': 'Walt Disney Productions',
 'Distributed by': 'Buena Vista Distribution',
 'Release date': ['January 25, 1961 ( 1961-01-25 )'],
 'Running time': '79 minutes',
 'Country': 'United States',
 'Language': 'English',
 'Budget': '$3.6 million',
 'Box office': '$303 million',
 'Running time (int)': 79,
 'Budget (float)': 3600000.0,
 'Box office (float)': 303000000.0,
 'Release date (datetime)': 'January 25, 1961',
 'imdb': '7.3',
 'metascore': '83',
 'rotten_tomatoes': '98%'}

In [193]:
save_data("disney_data_final.json", movie_info_copy)

#### Convert and save onto a CSV

In [198]:
import pandas as pd

df = pd.DataFrame(movie_info_list)

In [200]:
df.head(20)

,title,Production company,Release date,Running time,Country,Language,Box office,Running time (int),Budget (float),Box office (float),...,Adaptation by,Animation by,Traditional,Simplified,Music,Lyrics,Book,Basis,Productions,Awards
0,Academy Award Review of Walt Disney Cartoons,Walt Disney Productions,"[May 19, 1937 ( 1937-05-19 )]",41 minutes (74 minutes 1966 release),United States,English,$45.472,41.0,NaN,4.547200e+01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Snow White and the Seven Dwarfs,Walt Disney Productions,"[December 21, 1937 ( 1937-12-21 ) ( Carthay Ci...",83 minutes,United States,English,$418 million,83.0,1490000.0,4.180000e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Pinocchio,Walt Disney Productions,"[February 7, 1940 ( 1940-02-07 ) ( Center Thea...",88 minutes,United States,English,$164 million,88.0,2600000.0,1.640000e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fantasia,Walt Disney Productions,"[November 13, 1940 ( 1940-11-13 )]",126 minutes,United States,English,$76.4–$83.3 million (United States and Canada),126.0,2280000.0,8.330000e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,The Reluctant Dragon,Walt Disney Productions,"[June 27, 1941 ( 1941-06-27 )]",74 minutes,United States,English,"$960,000 (worldwide rentals)",74.0,600000.0,9.600000e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Dumbo,Walt Disney Productions,"[October 23, 1941 ( 1941-10-23 ) (New York Cit...",64 minutes,United States,English,>$1.3 million (est. United States/Canada renta...,64.0,950000.0,1.300000e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Bambi,Walt Disney Productions,"[August 9, 1942 ( 1942-08-09 ) (World Premiere...",70 minutes,United States,English,$267.4 million,70.0,858000.0,2.674000e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Saludos Amigos,Walt Disney Productions,"[August 24, 1942 ( 1942-08-24 ) (World Premier...",42 minutes,United States,NaN,$1.135 million (worldwide rentals),42.0,NaN,1.135000e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Victory Through Air Power,Walt Disney Productions,"[July 17, 1943 ( 1943-07-17 )]",70 min,United States,English,"$799,000",70.0,788000.0,7.990000e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,The Three Caballeros,Walt Disney Productions,"[December 21, 1944 ( 1944-12-21 ) (Mexico City...",71 minutes,United States,NaN,$3.355 million (worldwide rentals),71.0,NaN,3.355000e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [201]:
df.to_csv("disney_movie_data_final.csv")

In [202]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 466 entries, 0 to 465
Data columns (total 43 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   title                    466 non-null    object        
 1   Production company       194 non-null    object        
 2   Release date             459 non-null    object        
 3   Running time             452 non-null    object        
 4   Country                  416 non-null    object        
 5   Language                 446 non-null    object        
 6   Box office               389 non-null    object        
 7   Running time (int)       452 non-null    float64       
 8   Budget (float)           297 non-null    float64       
 9   Box office (float)       382 non-null    float64       
 10  Release date (datetime)  454 non-null    datetime64[ns]
 11  imdb                     452 non-null    object        
 12  metascore                452 non-nul

In [210]:
running_times = df.sort_values(['Box office (float)'], ascending=True)
running_times.head(20)

,title,Production company,Release date,Running time,Country,Language,Box office,Running time (int),Budget (float),Box office (float),...,Adaptation by,Animation by,Traditional,Simplified,Music,Lyrics,Book,Basis,Productions,Awards
413,Penguins,NaN,"[April 17, 2019 ( 2019-04-17 ) (United States)]",76 minutes,United States,English,$7.7\n million,76.0,NaN,7.700,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
133,Pete's Dragon,Walt Disney Productions,"[November 3, 1977 ( 1977-11-03 )]",128 minutes,United States,English,$36 –39.6 million,128.0,10000000.0,36.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Academy Award Review of Walt Disney Cartoons,Walt Disney Productions,"[May 19, 1937 ( 1937-05-19 )]",41 minutes (74 minutes 1966 release),United States,English,$45.472,41.0,NaN,45.472,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
343,The Boys: The Sherman Brothers' Story,NaN,"[April 24, 2009 ( 2009-04-24 ) (San Francisco ...",101 minutes,United States,English,"$55,513",101.0,NaN,55513.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
315,Morning Light,Walt Disney Pictures,"October 17, 2008 ( 2008-10-17 )",100 minutes,United States,English,"$275,776",100.0,NaN,275776.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
275,America's Heart and Soul,NaN,"[July 2, 2004 ( 2004-07-02 )]",84 minutes,United States,English,"$314,402",84.0,NaN,314402.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
238,Whispers: An Elephant's Tale,Walt Disney Pictures,"[March 10, 2000 ( 2000-03-10 )]",72 minutes,United States,English,"$500,000 (USA) (30 November 2000)",72.0,4000000.0,500000.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Victory Through Air Power,Walt Disney Productions,"[July 17, 1943 ( 1943-07-17 )]",70 min,United States,English,"$799,000",70.0,788000.0,799000.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,The Reluctant Dragon,Walt Disney Productions,"[June 27, 1941 ( 1941-06-27 )]",74 minutes,United States,English,"$960,000 (worldwide rentals)",74.0,600000.0,960000.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,The Sword and the Rose,Walt Disney Productions,"[July 23, 1953 ( 1953-07-23 ) ('US )]",92 minutes,NaN,English,$1 million (US),92.0,2000000.0,1000000.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [211]:
df['Profit'] = df['Box office (float)'] - df['Budget (float)']

In [229]:
df.head(10)

,title,Production company,Release date,Running time,Country,Language,Box office,Running time (int),Budget (float),Box office (float),...,Animation by,Traditional,Simplified,Music,Lyrics,Book,Basis,Productions,Awards,Profit
0,Academy Award Review of Walt Disney Cartoons,Walt Disney Productions,"[May 19, 1937 ( 1937-05-19 )]",41 minutes (74 minutes 1966 release),United States,English,$45.472,41.0,NaN,4.547200e+01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Snow White and the Seven Dwarfs,Walt Disney Productions,"[December 21, 1937 ( 1937-12-21 ) ( Carthay Ci...",83 minutes,United States,English,$418 million,83.0,1490000.0,4.180000e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,416510000.0
2,Pinocchio,Walt Disney Productions,"[February 7, 1940 ( 1940-02-07 ) ( Center Thea...",88 minutes,United States,English,$164 million,88.0,2600000.0,1.640000e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,161400000.0
3,Fantasia,Walt Disney Productions,"[November 13, 1940 ( 1940-11-13 )]",126 minutes,United States,English,$76.4–$83.3 million (United States and Canada),126.0,2280000.0,8.330000e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,81020000.0
4,The Reluctant Dragon,Walt Disney Productions,"[June 27, 1941 ( 1941-06-27 )]",74 minutes,United States,English,"$960,000 (worldwide rentals)",74.0,600000.0,9.600000e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,360000.0
5,Dumbo,Walt Disney Productions,"[October 23, 1941 ( 1941-10-23 ) (New York Cit...",64 minutes,United States,English,>$1.3 million (est. United States/Canada renta...,64.0,950000.0,1.300000e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,350000.0
6,Bambi,Walt Disney Productions,"[August 9, 1942 ( 1942-08-09 ) (World Premiere...",70 minutes,United States,English,$267.4 million,70.0,858000.0,2.674000e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,266542000.0
7,Saludos Amigos,Walt Disney Productions,"[August 24, 1942 ( 1942-08-24 ) (World Premier...",42 minutes,United States,NaN,$1.135 million (worldwide rentals),42.0,NaN,1.135000e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Victory Through Air Power,Walt Disney Productions,"[July 17, 1943 ( 1943-07-17 )]",70 min,United States,English,"$799,000",70.0,788000.0,7.990000e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11000.0
9,The Three Caballeros,Walt Disney Productions,"[December 21, 1944 ( 1944-12-21 ) (Mexico City...",71 minutes,United States,NaN,$3.355 million (worldwide rentals),71.0,NaN,3.355000e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [222]:
cols = df.columns.tolist()

In [223]:
cols = cols[-1:] + cols[:-1]

In [224]:
cols

['Profit',
 'title',
 'Production company',
 'Release date',
 'Running time',
 'Country',
 'Language',
 'Box office',
 'Running time (int)',
 'Budget (float)',
 'Box office (float)',
 'Release date (datetime)',
 'imdb',
 'metascore',
 'rotten_tomatoes',
 'Directed by',
 'Written by',
 'Based on',
 'Produced by',
 'Starring',
 'Music by',
 'Distributed by',
 'Budget',
 'Story by',
 'Narrated by',
 'Cinematography',
 'Edited by',
 'Languages',
 'Screenplay by',
 'Countries',
 'Production companies',
 'Color process',
 'Japanese',
 'Hepburn',
 'Adaptation by',
 'Animation by',
 'Traditional',
 'Simplified',
 'Music',
 'Lyrics',
 'Book',
 'Basis',
 'Productions',
 'Awards']

In [225]:
cols = [
 'title',
 'Production company',
 'Release date',
 'Running time',
 'Country',
 'Language',
 'Box office',
 'Running time (int)',
 'Budget (float)',
 'Box office (float)',
 'Profit',
 'Release date (datetime)',
 'imdb',
 'metascore',
 'rotten_tomatoes',
 'Directed by',
 'Written by',
 'Based on',
 'Produced by',
 'Starring',
 'Music by',
 'Distributed by',
 'Budget',
 'Story by',
 'Narrated by',
 'Cinematography',
 'Edited by',
 'Languages',
 'Screenplay by',
 'Countries',
 'Production companies',
 'Color process',
 'Japanese',
 'Hepburn',
 'Adaptation by',
 'Animation by',
 'Traditional',
 'Simplified',
 'Music',
 'Lyrics',
 'Book',
 'Basis',
 'Productions',
 'Awards']

In [226]:
cols

['title',
 'Production company',
 'Release date',
 'Running time',
 'Country',
 'Language',
 'Box office',
 'Running time (int)',
 'Budget (float)',
 'Box office (float)',
 'Profit',
 'Release date (datetime)',
 'imdb',
 'metascore',
 'rotten_tomatoes',
 'Directed by',
 'Written by',
 'Based on',
 'Produced by',
 'Starring',
 'Music by',
 'Distributed by',
 'Budget',
 'Story by',
 'Narrated by',
 'Cinematography',
 'Edited by',
 'Languages',
 'Screenplay by',
 'Countries',
 'Production companies',
 'Color process',
 'Japanese',
 'Hepburn',
 'Adaptation by',
 'Animation by',
 'Traditional',
 'Simplified',
 'Music',
 'Lyrics',
 'Book',
 'Basis',
 'Productions',
 'Awards']

In [227]:
df_copy = df[cols]

In [228]:
df_copy.head()

,title,Production company,Release date,Running time,Country,Language,Box office,Running time (int),Budget (float),Box office (float),...,Adaptation by,Animation by,Traditional,Simplified,Music,Lyrics,Book,Basis,Productions,Awards
0,Academy Award Review of Walt Disney Cartoons,Walt Disney Productions,"[May 19, 1937 ( 1937-05-19 )]",41 minutes (74 minutes 1966 release),United States,English,$45.472,41.0,NaN,4.547200e+01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Snow White and the Seven Dwarfs,Walt Disney Productions,"[December 21, 1937 ( 1937-12-21 ) ( Carthay Ci...",83 minutes,United States,English,$418 million,83.0,1490000.0,4.180000e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Pinocchio,Walt Disney Productions,"[February 7, 1940 ( 1940-02-07 ) ( Center Thea...",88 minutes,United States,English,$164 million,88.0,2600000.0,1.640000e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fantasia,Walt Disney Productions,"[November 13, 1940 ( 1940-11-13 )]",126 minutes,United States,English,$76.4–$83.3 million (United States and Canada),126.0,2280000.0,8.330000e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,The Reluctant Dragon,Walt Disney Productions,"[June 27, 1941 ( 1941-06-27 )]",74 minutes,United States,English,"$960,000 (worldwide rentals)",74.0,600000.0,9.600000e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [230]:
df = df[cols]

In [233]:
df.['Profit']

SyntaxError: invalid syntax (<ipython-input-233-8074c56bc9af>, line 1)